# 작동 원리

'이런문장을띄어쓰기한다고생각해보자'

**우리는 위의 문장을 어떻게 띄어쓸까?** 아마 대부분 (1) '이런 문장을 띄어쓰기 한다고 생각 해 보자' 혹은 (2) '이런 문장을 띄어쓰기 한다고 생각해 보자' 정도로 띄어쓸 것이다. 

그럼 한 가지 더 다시 실험해보자. 앞에서부터 위 문장을 읽지 말고 뒷부분의 '다고생각해보'를 먼저 쳐다보면 (좀 부자연스럽지만) '다고 생각해보'라고 띄울 수 있다. 그렇다면 조금 더 넓게 '한다고생각해보자'를 읽으면 확신을 가지고서 '한다고 생각해보자'라고 [-다고] 다음을 띄울 것이다. 무슨 말인고 하니, 한 부분을 띄울지 판단하기 위해서 어느 정도 맥락만 파악할 수 있다면 local하게 봐도 됨을 의미한다. 그리고 그 맥락이 더 명확할수록 더 자신있게 띄울 수 있을 것이다. 

이 말을 좀 더 정리하면 '한다고 생각해보자'로 띄울지 판단하기 위해서 몇 개의 features를 만들 수 있다. 

\# Left-side features  (L)

    c[-2:0]: 본인 글자 c[0]와 앞의 2개 글자
    c[-3:0]: 본인 글자 c[0]와 앞의 3개 글자
    ...
    c[-6:0]: 본인 글자 c[0]와 앞의 6개 글자


\# Central features    (C)

    c[-1:1]: 앞 뒤 1개 글자
    c[-2:1]: 앞 2글자, 뒤 1글자
    c[-1:2]: 앞 1글자, 뒤 2글자
    ...
    c[-1:5]: 앞 1글자, 뒤 5글자


\# Right-side features (R)

    c[0:2] : 본인 글자 c[0]와 뒤의 3개 글자
    ...
    c[0:6] : 본인 글자 c[0]와 뒤의 6개 글자

만약 아래와 같이 각 C[-n:m]에 대해서 띄어쓰기가 된 걸 봤다면, 당신은 어떤 선택을 하겠는가? 지금까지 본 데이터에서 '한다고' 다음에 80번이나 띄웠고 3번 븥여 썼다면, 80/(80 + 3) 점 정도로 띄어쓰기를 한다고 생각할 수 있다. 

같은 방법으로 붙여쓰기 점수도 3 / (80 + 3) 로 계산할 수 있다. 띄어쓴다/붙여쓴다 방향도 표현하기 위해서 점수의 폭을 [-1, 1]로 하자. -1점이면 반드시 붙인다. 1점이면 반드시 띈다. 0점이면 정말 모른다. 라는 의미이다. 

하지만 띄어쓰기를 판단할 글자의 왼쪽이나 오른쪽 부분만을 살펴보면 잘못된 판단을 할 수 있다. 

'C[-3:0] = 지금나가'

-가 다음에 띄어쓸 가능성이 매우 높아 보인다. 하지만 오른쪽의 한 글자를 더 보니 'C[-3:1] = 지금나가요' 였다면? 그렇기 때문에 Central features를 살펴보는게 가장 좋다.

만약 L, C, R 세 개의 features에서 모두 다 띄어쓰기 점수가 (0.9, 0.8, 0.85)라면 당연히 그 글자를 띈다고 판단해도 되지 않을까? 

L이 3번 띄고 0번 안띈 경우가 발생해도 점수는 3 / (3 + 0) = 1점이다. 노이즈에 민감할 수 있기 때문에 최소 p번 (parameter: min_count) 나온 경우에만 L, C, R의 features로 인정하고, min_count 이하로 나온 경우에는 0점을 주도록 하자. 

그러나 L, C, R 모두가 0점이 아닌 경우는 매우 드물 것이다. 하지만 앞의 예제에서 보았듯이 좌/우의 문맥을 모두 살펴보면 잘못된 판단을 할 가능성이 적다. 이러한 모든 상황들을 고려하여 아래와 같은 **띄어쓰기 판단 규칙**을 정하였다. 아래에 등장하는 threshold 들은 모두 모델의 패러매터로 들어간다.

한 글자의 띄어쓰기를 판단하는 규칙은 위와 같다. 그럼 어떤 글자들부터 띄어쓰기 판단을 해야 할까? Conditional Random Field, Hidden Markov Model 기반 알고리즘들은 좌 --> 우, 혹은 우 --> 좌 중 한 방향으로 전진/후진하며 순차적으로 띄어쓰기를 판단한다. (sequential labeling)

하지만 더 합리적인 방법은 확실한 경우부터 띄어쓰기를 판단하는 것 아닐까? 이 방법은 sequential labeling 보다 error propagation의 위험이 훨씬 적다. 앞의 예제에서 'c[-3:0] = 기한다고'를 판단하기 전에 'c[-3:2] = 띄어쓰기한다'에 대하여 '띄어쓰기 한다'로 띄어쓴다고 판단을 했다면, 'c[-3:0] = 기한다고'를 판단할 때, 0000은 이전 단계에서 태깅한 정보와 일치하지 않기 때문에 features로 고려하지 않는다. c[-3:0] 일 때 c[0] 다음에 띄어쓸 점수가 15 / (15 + 3) --> 15 / 15으로 높아진다. 즉 **이전 단계에서 확실한 부분들을 태깅하기 때문에, 주변의 애매모호한 features 중 유효한 features만 남겨준다**. 

그렇다면 iteration을 돌면서 한 글자씩 띄어쓰기를 개선한다는 의미가 된다. 이는 문장이 길어질수록 띄어쓰기 오류 수정 시간을 길게 만든다. 하지만 만약 띄어쓰기 점수의 크기가 force_abs_threshold 보다 크다면 점수 크기 순서와 관계없이 그 자리에서 띄어쓰기 태깅을 수행하자. 확실한 글자들을 굳이 다음 iteration으로 미루지 말자는 것이다. 우리는 이것을 **force tag**라 부르고, nonspace_threshold 보다 작거나 space_threshold 보다 큰 글자 중에서 가장 점수 크기가 큰 글자를 태깅하는 것을 **sequential tag**라 부른다. 

여기에 더해서 데이터 관계없이 반드시 이 글자들은 띈다, 붙인다라고 판단하는 글자도 있을 수 있고, 이 글자들은 미리 띄어쓰기 태깅을 해두는 것이 좋다. 이를 **rule tag**라고 부른다. 

또 한가지! 띄어쓰기 오류에 대해서 정의를 해보자. 사람들은 띄어쓰기를 안해야 할 걸 띄어쓸까? '데이 터를만 지는직 업' 이란 문장을 보면 많이 불편하다. 띄우지 않을 글자를 띄우면 사람도 읽기 어렵다. 그렇기 때문에 이런 오류를 사람들은 잘 발생시키지 않는다 (애초에 내가 못 읽으니까) 하지만 '데이터를만지는직업'이라고 모두 붙여써도 우리는 잘 읽을 수 있다. 그래서 사람들은 띄어야 할 것을 붙여 쓰는 오류를 쉽게 일으킨다. 즉 우리가 수정하고 싶은 오류는 **띄어야 할 것을 붙여 쓴 경우**이다. 그리고 input에 띄어쓰기가 되어 있다면, 이는 사람이 태깅해준 매우 유용한 띄어쓰기 태그 '1'이다. 이 부분은 알고리즘이 수정하지 않도록 했다. 

사용법은 아래와 같다. 

# Load and apply model

git clone을 하였을 경우에는 git repository를 입력하십시요

In [ ]:
import sys

YOUR_SOY_GIT_REPOSITORY = '../'
sys.path.append('%s' % YOUR_SOY_GIT_REPOSITORY)

In [1]:
from soyspacing.countbase import RuleDict, CountSpace

# Training spacing model

- 모델을 학습시킬 데이터는 preprocessor의 normalizer에 의하여 유효한 글자열만을 학습함
- 어떤 형태던지 상관없음 (normalizer가 유의미한 글자만 걸러내기 때문에)

- num_lines는 fname 파일의 num_lines 번째 줄까지만 학습함 (디버깅용)
- num_lines=-1 (default)이면 모든 줄을 다 학습함

    model.train(fname='YOUR_CORPUS_FILE', num_lines=-1)
    
    
- 이전 버전에서 json format으로 모델을 저장하였지만, 로딩 시간 단축을 위해서 text 형식으로만 저장을 만듦. json format으로 모델을 로딩할 때에는 아래의 옵션
    
    model.load_model(fname='YOUR_FILE', json_format=True) 
    

- text 형식으로 모델을 로딩할 때에는 아래의 옵션으로!

    model.load_model(fname='YOUR_FILE', json_format=False)
    

- 모델을 세이브하면 이제는 text 포멧으로만 모델을 저장함


기본적으로 데이터의 띄어쓰기가 잘 되어 있으면 띄어쓰기 오류를 더 잘 수정할 것이다. 우리는 앞서 띄어쓰기의 오류란, '띄어야 할 것은 띄우지 않은 경우'라고 정의하였다. 즉 어떤 문장에 33글자가 있는데 3번 빈칸이 있다면, 36글자가 있는데 6번 빈칸이 있는 경우보다 띄어쓰기 오류가 더 많다고 짐작할 수 있다. 그래서 글자 대비 빈칸의 비율을 계산한 뒤, 빈칸의 비율이 높은 순으로 0등 100등까지 등수를 부여하였다. 즉 등수가 0 ~ 1인 글은 띄어쓰기가 잘 되어있는 상위 1% 학습데이터라고 짐작할 수 있다. 

In [2]:
corpus_fname = '../demo_model/134963_norm.txt'
model = CountSpace()
model.train(corpus_fname)

all tags length = 694236 --> 57795, (num_doc = 15602)

In [12]:
model_fname = '../demo_model/test.model'
model.save_model(model_fname, json_format=False)

model2 = CountSpace()
model2.load_model('../demo_model/test.model', json_format=False)

### Load model and rules

- 아래와 같이 모델을 중복으로 로딩할 수 있음
    - 중복으로 로딩될 때에는 (chars,tags)의 빈도수가 추가되는 것

In [3]:
another_model_fname = '../demo_model/test.model'
model.load_model(another_model_fname, json_format=False)

In [4]:
rule_fnames = ['../demo_model/rules_token_2.txt', '../demo_model/rules_token_3.txt']
min_rule_length = 2
max_rule_length = 3

rule_dict = RuleDict(min_rule_length, max_rule_length, rule_fnames)

../demo_model/rules_token_2.txt does not exist
../demo_model/rules_token_3.txt does not exist


- RuleDict 안에는 chars: tags가 들어있음
    - chars: str type
    - tags: tuple type


- rules는 룰 단어 길이보다 1이 더 긴 태그 정보가 들어가야 함. 
    - 룰 단어 앞에를 띄우는지 (1), 안띄우는지 (0)를 표시해야 함
    - 불확실한 경우에는 룰 파일에 ?라고 기록하면 None으로 바꿔서 로딩
        
        - e.g: 지수가 ?001 -> "지수가": (None, 0, 0, 1)


- min_rule_length, max_rule_length를 정해줘서 rule tag를 빠르게 할 수 있게 함
    - 아래의 경우에는 min_rule_length=2, max_rule_length=3


    {
     '지수가': (1, 0, 0, 1),
     '전화는': (1, 0, 0, 1),
     '지랄마': (1, 0, 0, 1),
     '이때가': (1, 0, 0, 1),
     '골라놔': (1, 0, 0, 1),
     '별별' : (1, 0, 1),
     '정수빈': (1, 0, 0, 1),
     '주일날': (1, 0, 0, 1),
     '있든데': (1, 0, 0, 1),
     '쓸말이': (1, 0, 0, 1),
     '잘해쩡': (1, 0, 0, 1),
     '크니까': (1, 0, 0, 1),
     ...
    } 

## 띄어쓰기 교정 함수 적용

- arguments

    - 4개의 parameter
        - force_abs_threshold: 점수의 절대값이 이 수준 이상이면 최고점이 아니더라도 즉각 태깅
        - nonspace_threshold : 이 점수 이하일 때만 0으로 태깅
        - space_threshold    : 이 점수 이상일 때만 1로 태깅
        - min_count          : L, C, R 각각의 feature 빈도수가 min_count 이하이면 불확실한 정보로 판단, 띄어쓰기 계산 시 무시
        
    - verbose: iteration 마다 띄어쓰기가 어떻게 되고 있는지 확인
    
    - rules  : 점수와 관계없이 반드시 태깅을 먼저 할 (chars, tags)
        

In [5]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

In [6]:
sent = '이건진짜좋은영화 라라랜드진짜좋은영화'

sent_corrected, tags = model.correct(doc=sent, verbose=verbose, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 이건진짜좋은영화 라라랜드진짜좋은영화
after : 이건 진짜 좋은 영화 라라랜드진짜 좋은 영화


- model.correct()의 결과물은 띄어쓰기가 수정된 문장과 각 글자의 태그가 출력됨
- 띄어쓰기를 해야할지 말지 모호한 경우에는 None으로 표시되며, 띄어쓰기 수정 시에는 None은 0과 함께 띄지 않음

In [7]:
tags

[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1]

- verbose = True 일 때는 각 단계마다 띄어쓰기 태그가 어떻게 수정되는지가 출력됨

In [8]:
model.correct(rules=rule_dict, doc=sent, verbose=True, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

Input: ? ? ? ? ? ? ? 1 ? ? ? ? ? ? ? ? ? 1 
이건진짜좋은영화 라라랜드진짜좋은영화
Ruled: ? ? ? ? ? ? ? 1 ? ? ? ? ? ? ? ? ? 1 
이건진짜좋은영화 라라랜드진짜좋은영화
Force tagged (iter=1): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 
Force tagged (iter=2): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 


('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

debug=True 일 때에는 각 글자의 feature 빈도수, score, lcr score를 출력, force tagging, sequential tagging 순서에 따라 출력

In [9]:
model.correct(rules=rule_dict, doc=sent, debug=True, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

0: 이 (-1.000, 32)	lcr = (0.000, 0.000, -1.000)
1: 건 (0.750, 48)	lcr = (0.000, 0.750, 0.750)
2: 진 (-1.000, 106)	lcr = (-1.000, -1.000, -1.000)
3: 짜 (0.703, 90)	lcr = (0.750, 0.655, 0.000)
4: 좋 (-1.000, 942)	lcr = (-1.000, 0.000, -1.000)
5: 은 (0.504, 1430)	lcr = (0.000, 0.403, 0.604)
6: 영 (-1.000, 1586)	lcr = (-1.000, -1.000, -1.000)
7: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
8: 라 (-0.998, 2030)	lcr = (-1.000, -1.000, -0.995)
9: 라 (-0.938, 2860)	lcr = (-0.882, -0.962, -0.968)
10: 랜 (-1.000, 2792)	lcr = (-1.000, -1.000, 0.000)
11: 드 (-0.211, 1840)	lcr = (-0.211, 0.000, 0.000)
12: 진 (-1.000, 58)	lcr = (0.000, 0.000, -1.000)
13: 짜 (0.655, 58)	lcr = (0.000, 0.655, 0.000)
14: 좋 (-1.000, 942)	lcr = (-1.000, 0.000, -1.000)
15: 은 (0.502, 1426)	lcr = (0.000, 0.403, 0.601)
16: 영 (-1.000, 1426)	lcr = (-1.000, -1.000, 0.000)
17: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
force tagging i=0, score=-1.000
force tagging i=1, score=0.750
force tagging i=2, score=-1.000
force tagging i=3, score=0.685
force

('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

space_rules.txt는 다섯가지 단어에 대하여 띄어쓰기 규칙을 hard coding 할 수 있도록 넣어둔 사전입니다. 

    진짜    101
    
위 줄은 '진짜'라는 단어의 앞/뒤는 반드시 띄어쓰라는 의미입니다. 아래 문장에서 '라라랜드진짜'의 경우, '진짜'라는 단어를 제대로 알지 못해서 띄어쓰기를 하지 못했습니다 (태그가 None, None입니다)

    이건 진짜 좋은 영화 라라랜드진짜 좋은 영화
    
알려진 단어/어절에 대해서는 이와 같이 띄어쓰기를 강제할 수 있습니다. 

In [11]:
cat ./space_rules.txt

가령	101
진짜	101
가게는	1001
가게로	1001
가게야	1001

In [10]:
rule_fnames = ['./space_rules.txt']
min_rule_length = 2
max_rule_length = 3

rule_dict = RuleDict(min_rule_length, max_rule_length, rule_fnames)
rule_dict.rule_dict

{'가게는': (1, 0, 0, 1),
 '가게로': (1, 0, 0, 1),
 '가게야': (1, 0, 0, 1),
 '가령': (1, 0, 1),
 '진짜': (1, 0, 1)}

In [12]:
model.correct(sent, verbose, mc, ft, nt, st, rules=rule_dict)

('이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1])